# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [6]:
# Load the data
file_path = Path('../LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Preprocessing

In [13]:
# Check for non numerical columns.
pd.set_option('display.max_rows', 100)
t = df.columns.to_series().groupby(df.dtypes).groups
t

{dtype('float64'): Index(['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
        'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
        'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
        'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
        'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
        'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code',
        'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
        'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
        'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
        'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
        'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
        'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
        'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
        'mort_acc', 'mths_since_r

In [14]:
# Make a list of columns to be encoded.

columns_toEncode=['home_ownership', 'verification_status', 'issue_d',
        'pymnt_plan', 'initial_list_status', 'next_pymnt_d', 'application_type',
        'hardship_flag', 'debt_settlement_flag']


In [15]:
# Import LabelEncoder for converting the datatype from object to to numeric.
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in columns_toEncode:
    
    df[col] = le.fit_transform(df[col])
    
df.head(5)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,1,2,low_risk,0,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0
1,25000.0,0.2000,929.09,1,105000.0,2,2,low_risk,0,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0
2,20000.0,0.2000,529.88,1,56000.0,2,2,low_risk,0,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0
3,10000.0,0.1640,353.55,3,92000.0,2,2,low_risk,0,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0
4,22000.0,0.1474,520.39,1,52000.0,0,2,low_risk,0,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0


In [16]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = df['loan_status']

In [46]:
# Import Scaler
from sklearn.preprocessing import StandardScaler

# Creating StandardScaler instance
scaler = StandardScaler()

# Fitting Standard Scaller
X_scaler = scaler.fit(X)

# Scaling the data.
X=X_scaler.transform(X)



In [47]:
# Check to make sure the coulmns are scaled.
import numpy as np
print(np.mean(X[:,0]))
print(np.std(X[:,0]))

9.148042836558536e-17
1.0


# Split the Data into Training and Testing

In [23]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,0.805542,0.0,21.778153,0.217766,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.714932,0.0,20.199244,0.718367,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.0,0.000000,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.000000,0.0,13.890000,0.000000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,1.000000,0.0,19.760000,0.000000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,1.000000,0.0,26.660000,0.000000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,2.000000,0.0,999.000000,18.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0


In [25]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [48]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,random_state=1)

In [49]:
#Check shape.
X_train.shape,y_train.shape

((51612, 85), (51612,))

In [50]:
# Check counter for y train.
Counter(y_train)

Counter({'low_risk': 51366, 'high_risk': 246})

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [51]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros=RandomOverSampler(random_state=1)
X_resampled,y_resampled=ros.fit_resample(X_train,y_train)

Counter(y_resampled)

Counter({'low_risk': 51366, 'high_risk': 51366})

In [9]:
# Resample the training data with the RandomOversampler


Counter({'low_risk': 51366, 'high_risk': 51366})

In [52]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=1, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)
model.fit(X_resampled,y_resampled)

LogisticRegression(random_state=1)

In [10]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=1, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [53]:
# Calculated the balanced accuracy score
y_pred=model.predict(X_test)

from sklearn.metrics import balanced_accuracy_score
bas=balanced_accuracy_score(y_test,y_pred)
bas


0.7274397049152997

In [11]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

0.7163908158823367

In [54]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[   70,    31],
       [ 4074, 13030]], dtype=int64)

In [12]:
# Display the confusion matrix
# YOUR CODE HERE

array([[   73,    28],
       [ 4960, 12144]])

In [55]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.69      0.76      0.03      0.73      0.52       101
   low_risk       1.00      0.76      0.69      0.86      0.73      0.53     17104

avg / total       0.99      0.76      0.69      0.86      0.73      0.53     17205



In [13]:
# Print the imbalanced classification report
# YOUR CODE HERE

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.72      0.71      0.03      0.72      0.51       101
   low_risk       1.00      0.71      0.72      0.83      0.72      0.51     17104

avg / total       0.99      0.71      0.72      0.82      0.72      0.51     17205



### SMOTE Oversampling

In [57]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train)

Counter(y_resampled)

Counter({'low_risk': 51366, 'high_risk': 51366})

In [ ]:
# Resample the training data with SMOTE
# YOUR CODE HERE

Counter({'low_risk': 51366, 'high_risk': 51366})

In [58]:
# Train the Logistic Regression model using the resampled data
model=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=1, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)
model.fit(X_resampled,y_resampled)

LogisticRegression(random_state=1)

In [ ]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=1, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [59]:
# Calculated the balanced accuracy score
y_pred=model.predict(X_test)

bas=balanced_accuracy_score(y_test,y_pred)
bas


0.7271766085635691

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

0.7001170474858525

In [62]:
# Display the confusion matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[   70,    31],
       [ 4083, 13021]], dtype=int64)

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

array([[   71,    30],
       [ 5178, 11926]])

In [63]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.69      0.76      0.03      0.73      0.52       101
   low_risk       1.00      0.76      0.69      0.86      0.73      0.53     17104

avg / total       0.99      0.76      0.69      0.86      0.73      0.53     17205



In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.70      0.70      0.03      0.70      0.49       101
   low_risk       1.00      0.70      0.70      0.82      0.70      0.49     17104

avg / total       0.99      0.70      0.70      0.82      0.70      0.49     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [65]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc=ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'high_risk': 246, 'low_risk': 246})

In [ ]:
# Resample the data using the ClusterCentroids resampler
# YOUR CODE HERE

Counter({'high_risk': 246, 'low_risk': 246})

In [74]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [ ]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=1, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [75]:
# Calculated the balanced accuracy score
y_pred=model.predict(X_test)

bas=balanced_accuracy_score(y_test,y_pred)
bas

0.7149572446720818

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

0.6433412021043078

In [68]:
# Display the confusion matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[   78,    23],
       [ 5596, 11508]], dtype=int64)

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

array([[  82,   19],
       [8983, 8121]])

In [69]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.77      0.67      0.03      0.72      0.52       101
   low_risk       1.00      0.67      0.77      0.80      0.72      0.51     17104

avg / total       0.99      0.67      0.77      0.80      0.72      0.51     17205



In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.81      0.47      0.02      0.62      0.40       101
   low_risk       1.00      0.47      0.81      0.64      0.62      0.37     17104

avg / total       0.99      0.48      0.81      0.64      0.62      0.37     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [76]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'high_risk': 51366, 'low_risk': 46641})

In [ ]:
# Resample the training data with SMOTEENN
# YOUR CODE HERE

Counter({'high_risk': 51361, 'low_risk': 46653})

In [78]:
# Train the Logistic Regression model using the resampled data
model=LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled,y_resampled)

LogisticRegression(random_state=1)

In [ ]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=1, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [79]:
# Calculated the balanced accuracy score
y_pred=model.predict(X_test)

bas=balanced_accuracy_score(y_test,y_pred)
bas

0.7149572446720818

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

0.69732081662329

In [81]:
# Display the confusion matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[   70,    31],
       [ 4501, 12603]], dtype=int64)

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

array([[   72,    29],
       [ 5443, 11661]])

In [84]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.69      0.74      0.03      0.71      0.51       101
   low_risk       1.00      0.74      0.69      0.85      0.71      0.51     17104

avg / total       0.99      0.74      0.69      0.84      0.71      0.51     17205



In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.71      0.68      0.03      0.70      0.49       101
   low_risk       1.00      0.68      0.71      0.81      0.70      0.48     17104

avg / total       0.99      0.68      0.71      0.81      0.70      0.48     17205

